In [1]:
import pandas as pd
import numpy as np
import random
random.seed(42)
from datetime import datetime, timedelta
from sklearn import preprocessing
%load_ext autoreload
%autoreload 2
from GBRT_for_TSF.utils import evaluate_with_xgboost
from mpmf.utils import get_top_1_motif, get_top_k_motifs, compute_point_after_average

# from sklearn.multioutput import MultiOutputRegressor
# import xgboost as xgb

In [2]:
# ================== Parameters ==================
include_covariates = True  # True/False : whether to include features or not
include_motif_information = 1  # 0: no motif info; 1: Top-1 Motif; 2: Top-K Motifs (Direct); 3: Top-K Motifs (Average); 4: Top-K Motifs (Weighted Average)
k_motifs = 1
no_points_after_motif = 1  # number of points to consider
include_itself = False
# ================================================



num_periods_output = 24  # to predict
num_periods_input = 24  # input

ALL_Test_Data = []
ALL_Test_Prediction = []

file_name = "traffic.npy"
data_path = r"../data/" + file_name
data = np.load(data_path)
data = data[0:90, :]
# data = data[0:3, :]

data = pd.DataFrame(data)

In [3]:
def New_preprocessing(TimeSeries):
    Data = []
    start_date = datetime(2012, 1, 1, 00, 00, 00)  # define start date
    for i in range(0, len(TimeSeries)):
        record = []
        record.append(TimeSeries[i])
        record.append(start_date.month)
        record.append(start_date.day)
        record.append(start_date.hour)
        record.append(start_date.weekday())
        record.append(start_date.timetuple().tm_yday)
        record.append(start_date.isocalendar()[1])
        start_date = start_date + timedelta(hours=1)
        Data.append(record)
    ########## change list of lists to df ################

    headers = [
        "traffic",
        "month",
        "day",
        "hour",
        "day_of_week",
        "day_of_year",
        "week_of_year",
    ]
    Data_df = pd.DataFrame(Data, columns=headers)
    sub = Data_df.iloc[:, 1:]
    # Normalize features to be from -0.5 to 0.5 as mentioned in the paper
    New_sub = preprocessing.minmax_scale(sub, feature_range=(-0.5, 0.5))
    Normalized_Data_df = pd.DataFrame(
        np.column_stack([Data_df.iloc[:, 0], New_sub]), columns=headers
    )
    if include_motif_information:
        if include_motif_information == 1: # get_top_1_motif
            df_motif = get_top_1_motif(
                TimeSeries, num_periods_output, l=no_points_after_motif, include_itself=include_itself
            )
            interested_features = [
                c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))
            ]
            df_motif = df_motif[interested_features]
        if include_motif_information == 2: # get_top_k_motifs (Direct)
            df_motif = get_top_k_motifs(
                TimeSeries, num_periods_output, k=k_motifs, l=no_points_after_motif, include_itself=include_itself
            )
            interested_features = [
                c for c in df_motif.columns if (("idx" not in c) and ("dist" not in c))
            ]
            df_motif = df_motif[interested_features]
        if include_motif_information == 3: # get_top_k_motifs (Unweighted Average)
            df_motif = get_top_k_motifs(
                TimeSeries, num_periods_output, k=k_motifs, l=no_points_after_motif, include_itself=include_itself
            )
            interested_features = [c for c in df_motif.columns if ("idx" not in c)]
            df_motif = df_motif[interested_features]
            df_motif = compute_point_after_average(df_motif)
        if include_motif_information == 4: # get_top_k_motifs (Weighted Average)
            df_motif = get_top_k_motifs(
                TimeSeries, num_periods_output, k=k_motifs, l=no_points_after_motif, include_itself=include_itself
            )
            interested_features = [c for c in df_motif.columns if ("idx" not in c)]
            df_motif = df_motif[interested_features]
            df_motif = compute_point_after_average(df_motif, method="weighted")   
        # Normailize motif features to be from -0.5 to 0.5
        New_df_motif = preprocessing.minmax_scale(df_motif, feature_range=(-0.5, 0.5))
        # Convert the numpy array back to a DataFrame using the original columns and index
        New_df_motif = pd.DataFrame(New_df_motif, columns=df_motif.columns, index=df_motif.index)
        Normalized_Data_df = pd.concat([Normalized_Data_df, New_df_motif], axis=1)

    #################################################################################################
    # cut training and testing training is 10392
    Train = Normalized_Data_df.iloc[0:10392, :]
    Train = Train.values
    Train = Train.astype("float32")
    Test = Normalized_Data_df.iloc[10392 - num_periods_input :, :]
    Test = Test.values
    Test = Test.astype("float32")
    # Number_Of_Features = 7
    Number_Of_Features = Normalized_Data_df.shape[1]

    ############################################ Windowing ##################################
    end = len(Train)
    start = 0
    next = 0
    x_batches = []
    y_batches = []
    # count = 0
    while next + (num_periods_input) < end:
        next = start + num_periods_input
        x_batches.append(Train[start:next, :])
        y_batches.append(Train[next : next + num_periods_output, 0])
        start = start + 1
    y_batches = np.asarray(y_batches)
    y_batches = y_batches.reshape(-1, num_periods_output, 1)
    x_batches = np.asarray(x_batches)
    x_batches = x_batches.reshape(-1, num_periods_input, Number_Of_Features)

    ############################################ Windowing ##################################
    end_test = len(Test)
    start_test = 0
    next_test = 0
    x_testbatches = []
    y_testbatches = []
    while next_test + (num_periods_input) < end_test:
        next_test = start_test + num_periods_input
        x_testbatches.append(Test[start_test:next_test, :])
        y_testbatches.append(Test[next_test : next_test + num_periods_output, 0])
        start_test = start_test + num_periods_input
    y_testbatches = np.asarray(y_testbatches)
    y_testbatches = y_testbatches.reshape(-1, num_periods_output, 1)
    x_testbatches = np.asarray(x_testbatches)
    x_testbatches = x_testbatches.reshape(-1, num_periods_input, Number_Of_Features)
    if include_covariates and include_motif_information:
        pass
    elif include_covariates and (not include_motif_information):
        pass
    elif (not include_covariates) and include_motif_information:
        selected_cols = np.r_[
            0, len(headers) : len(headers) + len(df_motif.columns.tolist())
        ]
        x_batches = x_batches[:, :, selected_cols]
        x_testbatches = x_testbatches[:, :, selected_cols]
    else: # (not include_covariates) and (not include_motif_information)
        pass
    # print(
    #     "x_batches shape:",
    #     x_batches.shape,
    #     "y_batches shape:",
    #     y_batches.shape,
    #     "x_testbatches shape:",
    #     x_testbatches.shape,
    #     "y_testbatches shape:",
    #     y_testbatches.shape,
    # )
    return x_batches, y_batches, x_testbatches, y_testbatches

In [4]:
x_batches_Full = []
y_batches_Full = []
X_Test_Full = []
Y_Test_Full = []
for i in range(0, len(data)):
    print("Processing Time Series:", i)
    x_batches = []
    y_batches = []
    X_Test = []
    Y_Test = []
    TimeSeries = data.iloc[i, :]
    x_batches, y_batches, X_Test, Y_Test = New_preprocessing(TimeSeries)
    for element1 in x_batches:
        x_batches_Full.append(element1)

    for element2 in y_batches:
        y_batches_Full.append(element2)

    for element5 in X_Test:
        X_Test_Full.append(element5)

    for element6 in Y_Test:
        Y_Test_Full.append(element6)

Processing Time Series: 0


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Processing Time Series: 1
Processing Time Series: 2
Processing Time Series: 3
Processing Time Series: 4
Processing Time Series: 5
Processing Time Series: 6
Processing Time Series: 7
Processing Time Series: 8
Processing Time Series: 9
Processing Time Series: 10
Processing Time Series: 11
Processing Time Series: 12
Processing Time Series: 13
Processing Time Series: 14
Processing Time Series: 15
Processing Time Series: 16
Processing Time Series: 17
Processing Time Series: 18
Processing Time Series: 19
Processing Time Series: 20
Processing Time Series: 21
Processing Time Series: 22
Processing Time Series: 23
Processing Time Series: 24
Processing Time Series: 25
Processing Time Series: 26
Processing Time Series: 27
Processing Time Series: 28
Processing Time Series: 29
Processing Time Series: 30
Processing Time Series: 31
Processing Time Series: 32
Processing Time Series: 33
Processing Time Series: 34
Processing Time Series: 35
Processing Time Series: 36
Processing Time Series: 37
Processing

In [5]:
xgboost_parameters = {
    "learning_rate": 0.2,
    "n_estimators": 800,
    "max_depth": 8,
    "min_child_weight": 1,
    "gamma": 0.0,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 1,
    "random_state": 42,
    "verbosity": 1, # 0=Silent, 1=Warning, 2=Info, 3=Debug
}

In [6]:
rmse, wape, mae, mape = evaluate_with_xgboost(
    num_periods_output,
    x_batches_Full,
    y_batches_Full,
    X_Test_Full,
    Y_Test_Full,
    xgboost_parameters,
    (include_covariates or (include_motif_information > 0)),
)

In [7]:
print("RMSE: ", rmse)
print("WAPE: ", wape)
print("MAE: ", mae)
print("MAPE: ", mape)

RMSE:  0.014734268
WAPE:  0.11111283
MAE:  0.00600007
MAPE:  0.12660703


In [8]:
import datetime

print(f"This notebook was last run end-to-end on: {datetime.datetime.now()}\n")
###
###
###
# RMSE:  0.013664559
# WAPE:  0.10890117
# MAE:  0.0058806404
# MAPE:  0.1256635



# include_covariates = False
# RMSE:  0.012148092
# WAPE:  0.10824086
# MAE:  0.0058449837
# MAPE:  0.13247783

# With motif information (include_itself = True)
# RMSE:  0.013024213
# WAPE:  0.1125414
# MAE:  0.006077212
# MAPE:  0.13631995

# With motif information (include_itself = False)
# RMSE:  0.012684238
# WAPE:  0.112486996
# MAE:  0.006074274
# MAPE:  0.13604957

This notebook was last run end-to-end on: 2025-12-26 17:35:04.100556

